<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/BLIP-2/AutoCaptioner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Connect runtime and mount Google Drive
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

from google.colab import drive
drive.mount('/content/drive')

Tesla T4, 15360 MiB, 15101 MiB
Mounted at /content/drive


In [ ]:
IMAGE_FOLDERS = '/content/drive/MyDrive/datasets/PinupFiles'

## Captioning pipeline

In [ ]:
#@markdown Install blip-2
from IPython.display import clear_output
def ret_directoryFileCount(dir_path):
    file_count = 0
    for path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, path)):
            file_count += 1
    return file_count
import sys
if 'google.colab' in sys.modules:
    !pip3 install salesforce-lavis
clear_output()
print('[1;32mDone! ✓')

Done! ✓


In [ ]:
#@markdown Init blip-2
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [ ]:
#@title Load the blip-2 model
#model, vis_processors, _ = load_model_and_preprocess(
#    name="blip2_opt", model_type="pretrain_opt2.7b", is_eval=True, device=device
#)
model, vis_processors, _ = load_model_and_preprocess(
    name="blip2_opt", model_type="caption_coco_opt2.7b", is_eval=True, device=device
)
vis_processors.keys()
clear_output(); print('[1;32mDone! ✓')

In [ ]:
#@markdown Functions
import os; import random; import json; import time;

def load_image(image_path, sizeDiv=8):
    raw_image = Image.open(f'{image_path}').convert('RGB')
    h, w = raw_image.size
    display(raw_image.resize(( int(h / sizeDiv) , int(w / sizeDiv) )))
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
    return image

def wrap_query(query):
    return "Question: " + query + " Answer:"

def write__logfile(file, msg):
    with open(file, 'a') as fa:
        fa.write(msg)
    return ''

def write_the_file(path, data_string):
    if len(str(data_string)):
        with open(path, 'w+') as fw:
            fw.write(str(data_string))
    else:
        if os.path.exists(path):
            os.remove(path)
    return '';

def ret__longest_val(arr):
    curr_v = ''
    curr_x = 0
    while arr:
        val = arr.pop(0)
        if len(val) >= curr_x:
            curr_x = len(val)
            curr_v = val
    return curr_v

def auto__get_AI_caption(IMG_SRC):
    image = load_image(IMG_SRC)
    cap1 = ret__longest_val( model.generate({"image": image}, use_nucleus_sampling=False, num_captions=4) )
    cap2 = ret__longest_val( model.generate({"image": image}, use_nucleus_sampling=True, num_captions=4) )
    cap3 = model.generate({"image": image, "prompt": wrap_query('What is out of the ordinary with this image?')}).pop(0)
    return [cap1, cap2, cap3]

def auto__create_caption(IMG_SRC, CAPTION_SRC):
    if os.path.isfile(CAPTION_SRC):
        return 1
    caption_array = auto__get_AI_caption(IMG_SRC)
    write_the_file(CAPTION_SRC, json.dumps(caption_array))
    return json.dumps(caption_array)

#caption = auto__get_AI_caption('/content/drive/MyDrive/datasets/PinupFiles/Beth-Lily/beth-lily-vol-1-set-1-1.jpg')
#write_the_file('/content/test.txt', caption)
#print(caption)
#image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

In [ ]:
#@title Optional: Unzipping all ZIP files in IMAGE_FOLDERS
#@markdown Unzip all folders in IMAGE_FOLDERS
from zipfile import ZipFile
import os

for item in os.listdir(IMAGE_FOLDERS):
    zips = os.path.join(IMAGE_FOLDERS, item)
    print(f'Extracting: {item}')
    with ZipFile(zips, 'r') as zObject:
        zObject.extractall(path=IMAGE_FOLDERS)

In [ ]:
#@title List directories and image count
import os
for item_name in os.listdir(IMAGE_FOLDERS):
    item = os.path.join(IMAGE_FOLDERS, item_name)
    if os.path.isdir(item):
        if "_captions" in item_name:
            continue;
        print( f'{item_name}: {ret_directoryFileCount(item)}' )

In [ ]:
#@title Captioning loop for all folders within IMAGE_FOLDERS
import os; import random; import json; import time;

LOGFILE_NAME = '/' + time.strftime("%Y%m%d-%H%M%S") + '.log'

for IMAGE_FOLDER in os.listdir(IMAGE_FOLDERS):
    FOLDER = os.path.join(IMAGE_FOLDERS, IMAGE_FOLDER)
    if os.path.isdir(FOLDER):
        if "_captions" in FOLDER:
            continue;
        os.makedirs(FOLDER + '_captions', exist_ok=True)
        for FILE_NAME in os.listdir(FOLDER):
            IMG_SRC = os.path.join(FOLDER, FILE_NAME)
            if os.path.isfile(IMG_SRC):
                f_name, f_ext = os.path.splitext(FILE_NAME)
                CAPTION_SRC = FOLDER + f'_captions/{f_name}.txt'
                caption = auto__create_caption(IMG_SRC, CAPTION_SRC)
                if caption == 1:
                    write__logfile(IMAGE_FOLDERS + LOGFILE_NAME, f'{f_name}: {caption}')
                else:
                    print(f'{f_name}: {caption}')

#    if IMAGE_FOLDER == 'Holly-Peers':
#        print('Found Holly-Peers')
#        break